In [3]:
# normalize_nifti_volumes.ipynb

import os
import nibabel as nib
import numpy as np
from tqdm import tqdm
import shutil

# Set your source and target directories
base_dir = r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\dataset'
input_dirs = ['train_clean', 'val_clean', 'test_clean']
output_dirs = ['train_norm', 'val_norm', 'test_norm']

modalities_to_normalize = ['PreT1', 'CeT1']

# Function to normalize image to [0, 1]
def min_max_normalize(volume):
    min_val = np.min(volume)
    max_val = np.max(volume)
    if max_val - min_val == 0:
        return np.zeros_like(volume)
    return (volume - min_val) / (max_val - min_val)

# Function to process a single subject folder
def process_subject(src_subject_path, dst_subject_path):
    for modality in os.listdir(src_subject_path):
        modality_path = os.path.join(src_subject_path, modality)
        if not os.path.isdir(modality_path):
            continue
        
        dst_modality_path = os.path.join(dst_subject_path, modality)
        os.makedirs(dst_modality_path, exist_ok=True)

        for file in os.listdir(modality_path):
            if not file.endswith(".nii.gz"):
                continue

            src_file_path = os.path.join(modality_path, file)
            img = nib.load(src_file_path)
            data = img.get_fdata()

            if modality in modalities_to_normalize:
                norm_data = min_max_normalize(data)
                norm_img = nib.Nifti1Image(norm_data, affine=img.affine, header=img.header)
                norm_filename = file.replace(".nii.gz", "_norm.nii.gz")
                nib.save(norm_img, os.path.join(dst_modality_path, norm_filename))
            else:
                # Copy non-normalized modalities (e.g., Segmentation) as-is
                shutil.copy2(src_file_path, os.path.join(dst_modality_path, file))

# Process all directories
for input_dir, output_dir in zip(input_dirs, output_dirs):
    src_path = os.path.join(base_dir, input_dir)
    dst_path = os.path.join(base_dir, output_dir)
    os.makedirs(dst_path, exist_ok=True)

    print(f"📂 Processing {input_dir} -> {output_dir}")
    for subject in tqdm(os.listdir(src_path)):
        subject_src_path = os.path.join(src_path, subject)
        subject_dst_path = os.path.join(dst_path, subject)
        os.makedirs(subject_dst_path, exist_ok=True)

        process_subject(subject_src_path, subject_dst_path)

print("✅ Normalization complete. Normalized data saved in *_norm.nii.gz format.")


📂 Processing train_clean -> train_norm


  0%|          | 0/55 [00:00<?, ?it/s]

100%|██████████| 55/55 [03:53<00:00,  4.24s/it]


📂 Processing val_clean -> val_norm


100%|██████████| 48/48 [00:59<00:00,  1.23s/it]


📂 Processing test_clean -> test_norm


100%|██████████| 55/55 [01:26<00:00,  1.57s/it]

✅ Normalization complete. Normalized data saved in *_norm.nii.gz format.
